In [1]:
import numpy as np
from icecream import ic
from numpy import *
import random
import copy
import tqdm.auto
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

c:\Users\david\OneDrive\Desktop\Project\Computational Intellingence\professor_code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset and Validation

In [2]:
problem = np.load('problem_4.npz')
x = problem['x']
y = problem['y']
print(x.shape)
print(y.shape)

(2, 5000)
(5000,)


In [ ]:
#problem = np.load('problem_2.npz')
#x = problem['x']
#y = problem['y']
#print(x.shape)
#print(y.shape)

In [ ]:
#problem = np.load('problem_0_validation.npz')
#x_validation = problem['x']
#y_validation = problem['y']

## FUNCTIONS - only a subset of operators (maybe u will change)

we can divide 2 array in:
- all function
- only mathematical 

In [3]:
all_attributes = dir(np)

# Filter for mathematical functions (callables that aren't classes or non-math utilities)
math_functions = {
    attr: getattr(np, attr)
    for attr in all_attributes
    if callable(getattr(np, attr)) and isinstance(getattr(np, attr), np.ufunc) }

#### ALL FUNCTIONS
print(math_functions)


desired_categories = ["add", "subtract", "multiply", "divide",  "power",
                        "sin", "cos","log"]

#- I will produce a tree only this subset of operation, is only to have toy examples
math_functions = { key: value for key, value in math_functions.items() if key in desired_categories}
print(math_functions)

dictionary_func_count = {'abs': 1, 'absolute': 1, 'acos': 1, 'acosh': 1, 'add': 2, 'arccos': 1, 'arccosh': 1, 'arcsin': 1, 'arcsinh': 1, 'arctan': 1, 'arctan2': 2, 'arctanh': 1, 'asin': 1, 'asinh': 1, 'atan': 1, 'atan2': 2, 'atanh': 1, 'bitwise_and': 2, 'bitwise_count': 1, 'bitwise_invert': 1, 'bitwise_left_shift': 2, 'bitwise_not': 1, 'bitwise_or': 2, 'bitwise_right_shift': 2, 'bitwise_xor': 2, 'cbrt': 1, 'ceil': 1, 'conj': 1, 'conjugate': 1, 'copysign': 2, 'cos': 1, 'cosh': 1, 'deg2rad': 1, 'degrees': 1, 'divide': 2, 'divmod': 2, 'equal': 2, 'exp': 1, 'exp2': 1, 'expm1': 1, 'fabs': 1, 'float_power': 2, 'floor': 1, 'floor_divide': 2, 'fmax': 2, 'fmin': 2, 'fmod': 2, 'frexp': 1, 'gcd': 2, 'greater': 2, 'greater_equal': 2, 'heaviside': 2, 'hypot': 2, 'invert': 1, 'isfinite': 1, 'isinf': 1, 'isnan': 1, 'isnat': 1, 'lcm': 2, 'ldexp': 2, 'left_shift': 2, 'less': 2, 'less_equal': 2, 'log': 1, 'log10': 1, 'log1p': 1, 'log2': 1, 'logaddexp': 2, 'logaddexp2': 2, 'logical_and': 2, 'logical_not': 1, 'logical_or': 2, 'logical_xor': 2, 'matmul': 2, 'maximum': 2, 'minimum': 2, 'mod': 2, 'modf': 1, 'multiply': 2, 'negative': 1, 'nextafter': 2, 'not_equal': 2, 'positive': 1, 'pow': 2, 'power': 2, 'rad2deg': 1, 'radians': 1, 'reciprocal': 1, 'remainder': 2, 'right_shift': 2, 'rint': 1, 'sign': 1, 'signbit': 1, 'sin': 1, 'sinh': 1, 'spacing': 1, 'sqrt': 1, 'square': 1, 'subtract': 2, 'tan': 1, 'tanh': 1, 'true_divide': 2, 'trunc': 1, 'vecdot': 2}
math_functions_count = { key:value for key , value in dictionary_func_count.items() if key in desired_categories}
print(math_functions_count)

inverted_dict = {}
for key, value in math_functions_count.items():
    inverted_dict.setdefault(value, []).append(key)

print(inverted_dict)

{'abs': <ufunc 'absolute'>, 'absolute': <ufunc 'absolute'>, 'acos': <ufunc 'arccos'>, 'acosh': <ufunc 'arccosh'>, 'add': <ufunc 'add'>, 'arccos': <ufunc 'arccos'>, 'arccosh': <ufunc 'arccosh'>, 'arcsin': <ufunc 'arcsin'>, 'arcsinh': <ufunc 'arcsinh'>, 'arctan': <ufunc 'arctan'>, 'arctan2': <ufunc 'arctan2'>, 'arctanh': <ufunc 'arctanh'>, 'asin': <ufunc 'arcsin'>, 'asinh': <ufunc 'arcsinh'>, 'atan': <ufunc 'arctan'>, 'atan2': <ufunc 'arctan2'>, 'atanh': <ufunc 'arctanh'>, 'bitwise_and': <ufunc 'bitwise_and'>, 'bitwise_count': <ufunc 'bitwise_count'>, 'bitwise_invert': <ufunc 'invert'>, 'bitwise_left_shift': <ufunc 'left_shift'>, 'bitwise_not': <ufunc 'invert'>, 'bitwise_or': <ufunc 'bitwise_or'>, 'bitwise_right_shift': <ufunc 'right_shift'>, 'bitwise_xor': <ufunc 'bitwise_xor'>, 'cbrt': <ufunc 'cbrt'>, 'ceil': <ufunc 'ceil'>, 'conj': <ufunc 'conjugate'>, 'conjugate': <ufunc 'conjugate'>, 'copysign': <ufunc 'copysign'>, 'cos': <ufunc 'cos'>, 'cosh': <ufunc 'cosh'>, 'deg2rad': <ufunc 'deg

In [4]:
x_min = np.min(x)
x_max = np.max(x)
print("X-MIN  ", x_min , "      -X-MAX  " ,x_max)

X-MIN   -4.999705785571271       -X-MAX   4.999534886222849


## Generation Node - Generation Tree - Full/Grow 

Node Structure:
    
    -operator
    -left
    -right
    -value
    -MSE

In [5]:
class Node:
    def __init__(self, operator=None, left=None, right=None, value=None  ):
        self.operator = operator  # Operator function (e.g., np.add)
        self.left = left          # Left child (Node)
        self.right = right        # Right child (Node)
        self.value = value        # Constant or variable (if leaf)


    def is_leaf(self):
        return self.operator is None

    def __str__(self):   #### __str__
        if self.is_leaf():
            return str(self.value)
        elif( self.left !=None and self.right!=None  ) :
            return f"({self.left} {self.operator.__name__} {self.right})"
        elif( self.left !=None and self.right==None ):
            return f"({self.operator.__name__} {self.left})"
        elif( self.left ==None and self.right!=None ):
            return f"({self.operator.__name__} {self.right})"

    def compute(self, variables_values):                ## variables_values is a dictionary
        if self.is_leaf():                              ## If it's a leaf, return the value (either constant or variable)
            if isinstance(self.value, str):  # Variable
                return variables_values[self.value]
            else:  # Constant
                return self.value

        if(self.left!=None and self.right!=None):
            left_value = self.left.compute(variables_values) #### here we are define the computation 
            right_value = self.right.compute(variables_values) 
            return self.operator(left_value, right_value)
        elif( self.left!=None and self.right==None):
            left_value = self.left.compute(variables_values) #### here we are define the computation 
            return self.operator(left_value)
        elif( self.left==None and self.right!=None):              #### in theory this condition never happens
            right_value = self.right.compute(variables_values) #### in the above node, we go left or right and we recall again the function
            return self.operator(right_value)                  #### to have again this loop, the box in the box
    

    def print_attributes(self):
        print(f"Operator: {self.operator.__name__ if self.operator else 'None'}")
        print(f"Left: {self.left if self.left else 'None'}")
        print(f"Right: {self.right if self.right else 'None'}")
        print(f"Value: {self.value if self.value is not None else 'None'}")

In [6]:
#### invert this one cell below, i don't have the class node 
def generate_full_tree(depth=4, variables={"x0":0, "x1":0}):
    global math_functions , math_functions_count , x_min , x_max
    if depth == 0:  # Create a leaf
        if random.random() < 0.5:
            return Node(value=random.uniform(x_min, x_max))  # Random constant
        else:
            return Node(value=random.choice(list(variables.keys())))  # Variable
    # Create an internal node (operator)
    keys = list(math_functions.keys())
    random_key = random.choice(keys) 
    random_func = math_functions[random_key]
    random_count = math_functions_count[random_key]
    if(random_count==2 ):
        left = generate_full_tree(depth - 1, variables)
        right = generate_full_tree(depth - 1, variables)
    else:
        left  = generate_full_tree(depth - 1, variables)
        right = None               ##### we force the right node to have None attribute
    return Node(operator=random_func, left=left, right=right)

In [8]:
### DEBUG GROW TREE FOR COMPUTATION0
def generate_grow_tree(depth=3, variables={"x0":0, "x1":0}):
    global math_functions , math_functions_count, x_min , x_max

    if depth == 0 or (depth > 0 and random.random() < 0.2):  # Create a leaf
        if random.random() < 0.1:
            return Node(value=random.uniform(x_min, x_max))  # Random constant we have to decide what is the range of the random value 
        else:
            return Node(value=random.choice(list(variables.keys())))  # Variable

    # Create an internal node (operator)  ######### HERE WE ADD THE MSE TO CONTINUE THE GROW
    keys = list(math_functions.keys())
    random_key = random.choice(keys) 
    random_func = math_functions[random_key]
    random_count = math_functions_count[random_key]
    if(random_count==2 ):
        ###we cna use self on the 
        left = generate_grow_tree(depth - 1, variables)
        right = generate_grow_tree(depth - 1, variables)
    else:
        left  = generate_grow_tree(depth - 1, variables)
        right = None               ##### we force the right node to have None attribute
    
    return Node(operator=random_func, left=left, right=right)

## Computation TRIAL MSE - YOU CAN DECIDE TO RUN - NOT NECESSARY

In [10]:
variables_values = {
    "x0": 5,
    "x1": 3,}

full_tree_standard = generate_full_tree(depth=3 , variables = variables_values )
print(full_tree_standard)
y1 = full_tree_standard.compute(variables_values)
print("solution full_tree ",y1 )

print()
grow_tree_standard = generate_grow_tree(depth=3 , variables=variables_values)
print(grow_tree_standard)
y2 = grow_tree_standard.compute(variables_values)
print("solution grow_tree ",y2 )

((cos (log 0.6571016013857776)) subtract ((x1 add x0) subtract (-0.9396806244567477 subtract x0)))
solution full_tree  -13.026557691525994

(sin (log x1))
solution grow_tree  0.8905770416677471


## MSE Function

In [11]:
def MSE_train(Node ):
    global x , y 
    y_formula_generate =np.array([])
    for el in x.T:
        dictio = { f"x{e}": el[e].item() for e in range(len(el))     }
        out = Node.compute(dictio)
        y_formula_generate= np.append(y_formula_generate, out)

    scalar = 100*np.square(y-y_formula_generate).sum()/len(x)
    return (Node , str(Node) , scalar)

## Mutation Function

### aggiustare le mutazioni, non devo perdere il padre

In [12]:
#### PARENT SELECTION USING TOURNAMENT SELECTION
def tournament_selection(population, tournament_size=10):
    tournament_contestants = random.sample(population, tournament_size)
    tournament_contestants.sort(key=lambda x: x[2], reverse=False)
    return tournament_contestants[0]

In [13]:
#mutation_list = [point_mutation , permutation ]        [ 0.7 , 0.3 ]  
#recombination_list = [recombination]
def select_operation(mutation_prob=0.05, crossover_prob=0.95):
    global mutation_list, recombination_list

    rand = random.random()  # random number between 0 and 1
    if rand < mutation_prob:
        func= random.choices(mutation_list, weights=[0.4 ,0.2, 0.4], k=1)[0]
        return 1, func
    else:
        func= random.choices(recombination_list)
        return 2, func[0]

In [14]:
def get_node(node):
    parent_node= None
    branch =None
    obj= node
    ### mantenere parente e il nodo dove modificare

    while True:
        value = random.random()
        if(value < 0.7): #### here we go deep in the node 
            if( obj.right is not None and obj.left is not None   ):
                value = random.random()
                if(value < 0.5 ):
                    parent_node = obj
                    branch= "r"
                    obj=obj.right
                else:
                    parent_node = obj
                    branch= "l"
                    obj=obj.left
            elif( obj.right is None and obj.left is not None   ):
                parent_node = obj
                branch= "l"
                obj=obj.left
            
            elif(obj.right is not None and obj.left is None): 
                parent_node = obj
                branch= "r"
                obj=obj.right
            
            elif(obj.operator is None):
                break                       ### the node is a leaf
        else:
            break
            #same the node 

    return obj , parent_node ,branch

In [15]:
def hoist(obj_input):                                       ### WORKS
    global math_functions , math_functions_count, inverted_dict
    obj= copy.deepcopy(obj_input)
    first_obj = copy.deepcopy(obj_input)
    i=0   ####check which level i'm modifying
    obj1_extracted , parent_node ,branch = get_node(obj)    
    return obj1_extracted

In [16]:
def point_mutation(obj_input):                                            ### WORKS
    global math_functions , math_functions_count, inverted_dict
    obj= copy.deepcopy(obj_input)
    
    first_obj = copy.deepcopy(obj_input)
    i=0
    while True:
        if(i<=1000):
            value = random.random()
            if(value < 0.7): #### here we go deep in the node 
                if( obj.right is not None and obj.left is not None   ):
                    value = random.random()
                    if(value < 0.5 ):
                        obj=obj.right
                    else:
                        obj=obj.left

                elif( obj.right is None and obj.left is not None   ):
                    obj=obj.left
                
                elif(obj.right is not None and obj.left is None): 
                    obj=obj.right
                
                elif(obj.operator is None):
                    #break 
                    obj = obj_input
            else:
                if(obj.operator is not None and (  obj.right is not None and obj.left is not None ) ):  ### binary ope
                    list_ope = inverted_dict[2]
                    random_key = random.choice(list_ope) 
                    random_func = math_functions[random_key]
                    obj.operator= random_func
                    break

                elif(obj.operator is not None and ((  obj.right is not None and obj.left is None ) or ( obj.right is None and obj.left is not None   )  )  ):   ## unary operator
                    list_ope = inverted_dict[1]
                    random_key = random.choice(list_ope) 
                    random_func = math_functions[random_key]
                    obj.operator= random_func
                    break
                else:
                    obj = obj_input
        else:
            obj=first_obj
            break
        i+=1

    return  obj

In [17]:
#### WARNING THE PERMUTATION CAN BE ALSO FOR A NODE THAT HAS X0 divide X0, if we permutate that we get the same result
def permutation(obj_input):                                       ### WORKS
    global math_functions , math_functions_count, inverted_dict
    obj= copy.deepcopy(obj_input)

    first_obj = copy.deepcopy(obj_input)
    i=0   ####check which level i'm modifying
    while True:
        if(i<1000):
            if( (obj.right is not None and obj.left is not None)   ):
                if( obj.right.value is not None and obj.left.value  is not None ):
                    l= obj.left.value
                    r= obj.right.value
                    obj.right.value = l
                    obj.left.value = r
                    break
                else:
                    value = random.random()
                    if(value < 0.5 ):
                        obj=obj.right
                    else:
                        obj=obj.left

            elif( obj.right is None and obj.left is not None   ):
                obj=obj.left
            
            elif(obj.right is not None and obj.left is None): 
                obj=obj.right
            
            elif( obj.value is not None and ( obj.right is None and obj.left is None )):
                obj= first_obj
        else:
            obj=first_obj
            break
        i+=1
    return obj

In [18]:
def recombination(obj1 , obj2):                 ### WORKS                          
    global math_functions , math_functions_count, inverted_dict
    
    obj1_value= obj1
    obj2_value= obj2
    parent_node1 = None
    parent_node2 = None
    
    if(obj1_value.operator!= None ):
        while (parent_node1 is None and ( obj1_value.left is not None or  obj1_value.right is not None    )   ):
            obj1_extracted , parent_node1 ,branch1 = get_node(obj1_value)
    else:
        obj1_extracted = obj1_value
        parent_node1 = None
        branch1="root"
    
    extracted1 = copy.deepcopy(obj1_extracted)

    #print( "extracted to replace", id(obj1_extracted) , obj1_extracted)
    if(obj2_value.operator!= None ):
        while (parent_node2 is None and ( obj2_value.left is not None or  obj2_value.right is not None    ) ):
            obj2_extracted , parent_node2 ,branch2 = get_node(obj2_value)
    else:
        obj2_extracted = obj2_value
        parent_node2 = None
        branch2="root"
    #print("extracted to replace", id(obj2_extracted) , obj2_extracted)
    extracted2 = copy.deepcopy(obj2_extracted)

    if(branch1=="r"):
        parent_node1.right = extracted2
    elif(branch1=="l"):
        parent_node1.left = extracted2
    else:
        obj1_value = extracted2

    if(branch2=="r"):
        parent_node2.right = extracted1
    elif(branch2=="l"):
        parent_node2.left = extracted1
    else:
        obj2_value = extracted1
    return obj1_value, obj2_value

In [19]:
## grow_tree_standard  - full_tree_standard
print(grow_tree_standard)
print(full_tree_standard)

modified_grow_tree_standard = copy.deepcopy(grow_tree_standard)
modified_full_tree_standard = copy.deepcopy(full_tree_standard)

mut1, mut2 =recombination( modified_grow_tree_standard ,  modified_full_tree_standard)

print()
print(mut1)
print(mut2)
#new_y = 
#print()
#print( full_tree_standard )

(sin (log x1))
((cos (log 0.6571016013857776)) subtract ((x1 add x0) subtract (-0.9396806244567477 subtract x0)))

(sin 0.6571016013857776)
((cos (log (log x1))) subtract ((x1 add x0) subtract (-0.9396806244567477 subtract x0)))


In [20]:
mutation_list = [point_mutation , permutation, hoist ] 
recombination_list = [recombination]

## Implementation of GP

INITIALIZATION POP

### Generating pop

In [21]:
from collections import defaultdict


def filtered_POP(POP):

    float_value_counts = defaultdict(int)
    expression_counts = defaultdict(int)

    unique_strings = set()
    filtered_list = []

    for item in POP:
        node, expression, float_value = item

        # Check if the float value count is below the limit
        if float_value_counts[float_value] < 5 and expression_counts[expression] < 5:
            # Check if the string is unique
            if expression not in unique_strings:
                filtered_list.append(item)
                unique_strings.add(expression)
                float_value_counts[float_value] += 1
                expression_counts[expression] += 1
    return filtered_list

In [22]:
#### the variables_values is define by the number of row of x variable (matrix)
variables_values = { f"x{k}" : random.uniform(x_min, x_max) for k in range(len(x))}
variables_values

{'x0': np.float64(3.1211284430478834), 'x1': np.float64(4.704665666396284)}

In [23]:
POP = []               ### for medium problem 500

#variables_values = {
#    "x0": 5,
#    "x1": 3}

def generate_POP(POP):
    for i in range(500):
        if(i<250):
            full_tree_standard = generate_full_tree(depth=3 , variables=variables_values )
            y_formula_generate_grow =np.array([])
            for el in x.T:
                dictio = { f"x{e}": el[e].item() for e in range(len(el))     }
                out = full_tree_standard.compute(dictio)
                y_formula_generate_grow= np.append(y_formula_generate_grow, out)
                
            fitness = 100*np.square(y-y_formula_generate_grow).sum()/len(x)
            #print(fitness)
            if( not (np.isnan(fitness))):
                POP.append( (full_tree_standard, str(full_tree_standard) ,fitness) )
            else:
                POP.append( (full_tree_standard,str(full_tree_standard) , np.float64(np.inf)) )
        else:
            grow_tree = generate_grow_tree(depth=3 , variables=variables_values )
            y_formula_generate_grow =np.array([])
            for el in x.T:
                dictio = { f"x{e}": el[e].item() for e in range(len(el))     }
                out = grow_tree.compute(dictio)
                #print("input ", dictio , " value output ", out)
                y_formula_generate_grow= np.append(y_formula_generate_grow, out)
            fitness = 100*np.square(y-y_formula_generate_grow).sum()/len(x)
            #print(fitness)
            if( not (np.isnan(fitness))):
                POP.append((grow_tree, str(grow_tree), fitness))
            else:
                POP.append( (grow_tree, str(grow_tree), np.float64(np.inf)) )
        i+=1

    POP = sorted(POP, key=lambda x: x[2])
    seen = set()
    POP = [(obj, value1, value2) for obj, value1, value2 in POP if value1 not in seen and not seen.add(value1)]
    POP = sorted(POP, key=lambda x: x[2])


In [38]:
generate_POP(POP)
POP= filtered_POP(POP)
POP = sorted(POP, key=lambda x: x[2])

while len(POP)<100:
    generate_POP(POP)
    print(len(POP))
    POP= filtered_POP(POP)

POP = sorted(POP, key=lambda x: x[2])
for p in POP:
   print(p)

C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:32: RuntimeWarning: invalid value encountered in power
  return self.operator(left_value, right_value)
C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:35: RuntimeWarning: invalid value encountered in log
  return self.operator(left_value)
C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:32: RuntimeWarning: divide by zero encountered in power
  return self.operator(left_value, right_value)
C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:32: RuntimeWarning: overflow encountered in power
  return self.operator(left_value, right_value)
C:\Users\david\AppData\Local\Temp\ipykernel_6964\209081552.py:17: RuntimeWarning: overflow encountered in square
  fitness = 100*np.square(y-y_formula_generate_grow).sum()/len(x)
C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:35: RuntimeWarning: invalid value encountered in sin
  return self.operator(left_value)
C:\Users\david\AppData\Lo

(<__main__.Node object at 0x000002073975E590>, '((cos (x1 divide x1)) add ((4.327152256468638 subtract -4.451849440475967) power (cos x1)))', np.float64(1820743.7593726288))
(<__main__.Node object at 0x000002073912ACE0>, '((cos x1) add 1.6050490070955297)', np.float64(4058707.2431434034))
(<__main__.Node object at 0x0000020737DFFA90>, '((sin (cos x1)) subtract (log (cos 1.4184788461791227)))', np.float64(4156351.775243101))
(<__main__.Node object at 0x000002073912ACE0>, '(1.0418552103307164 add (sin (cos x1)))', np.float64(4450879.9165222775))
(<__main__.Node object at 0x000002073912ACE0>, '(0.9787261254650623 add (sin (cos x1)))', np.float64(4487228.476830467))
(<__main__.Node object at 0x000002073912ACE0>, '((sin (x1 add 1.0271478501807358)) add (sin (x1 divide x1)))', np.float64(4631021.909222898))
(<__main__.Node object at 0x000002073912ACE0>, '(1.8489550041656857 power (cos x1))', np.float64(4749049.778924013))
(<__main__.Node object at 0x000002073912ACE0>, '((cos -4.8991994428258

## parent selection

In [25]:
### OPEN STATE VARIABLE 

"""
import pickle

# Load the variable
with open("population_variable_problem_8.pkl", "rb") as f:
    POP = pickle.load(f)
"""

'\nimport pickle\n\n# Load the variable\nwith open("population_variable_problem_8.pkl", "rb") as f:\n    POP = pickle.load(f)\n'

In [50]:
MAX_STEPS=1_000
lenght_sample = int(len(POP)*0.20)                       ### 20% of the pop lenght
 
for steps in tqdm(range(MAX_STEPS)):
    off_spring = tournament_selection(POP,lenght_sample )
    off_spring_2 = tournament_selection(POP, lenght_sample)
    #print(off_spring)
    input_num, func = select_operation() ##### if pick the recombination - we get infinite loop
    if( input_num==2 ):             ### recombination - 2
        result_1 , result_2 = recombination( off_spring[0] , off_spring_2[0]  ) 
        
        nd1, ns_string1, v1 = MSE_train(result_1)
        #print("1 offstring", nd1, ns_string1, v1)
        if( np.isnan(v1)):
            v1 = np.float64(np.inf)

        if(ns_string1!=off_spring[1] and ns_string1!=off_spring_2[2] ):

            POP.append( ( nd1, ns_string1, v1) ) 
            POP = sorted(POP, key=lambda x: x[2])   
        

        nd2, ns_string2, v2 = MSE_train(result_2)
        if( np.isnan(v2)):
            v2 = np.float64(np.inf)
        #print("2 offstring", nd2, ns_string2, v2)

        if(ns_string2!=off_spring[1] and ns_string2!=off_spring_2[2] ):
            #print(ns_string1)
            POP.append( ( nd2, ns_string2, v2) ) 
            POP = sorted(POP, key=lambda x: x[2])   
        
    else:
        #print(func)
        result_1 =func( off_spring[0] )
        if(result_1 is not None):
            nd1, ns_string1, v1 = MSE_train(result_1) 
            if( np.isnan(v1)):
                v1 = np.float64(np.inf)
        if( str(off_spring[1])!=ns_string1 ):       #### avoid to add child with same fitness
            POP.append( ( nd1, ns_string1, v1)     ) 
            POP = sorted(POP, key=lambda x: x[2])
    ### filter the POP HAVING A UNIQUE ELEMENT
    POP = filtered_POP(POP)
    POP = POP[:500]

  6%|▌         | 59/1000 [00:05<01:34,  9.95it/s]C:\Users\david\AppData\Local\Temp\ipykernel_6964\3540341523.py:32: RuntimeWarning: invalid value encountered in power
  return self.operator(left_value, right_value)
100%|██████████| 1000/1000 [01:38<00:00, 10.16it/s]


In [52]:
for p in POP:
    print(p)

(<__main__.Node object at 0x000002073975E590>, '((cos (x1 divide x1)) add ((4.327152256468638 subtract -4.451849440475967) power (cos x1)))', np.float64(1820743.7593726288))
(<__main__.Node object at 0x000002073912ACE0>, '((cos x1) add 1.6050490070955297)', np.float64(4058707.2431434034))
(<__main__.Node object at 0x000002073975E590>, '(1.6050490070955297 add (cos x1))', np.float64(4058707.2431434034))
(<__main__.Node object at 0x0000020737DFFA90>, '((sin (cos x1)) subtract (log (cos 1.4184788461791227)))', np.float64(4156351.775243101))
(<__main__.Node object at 0x0000020737DFFA90>, '((cos x1) multiply 1.8489550041656857)', np.float64(4300039.647017371))
(<__main__.Node object at 0x000002073975E590>, '(4.327152256468638 add ((cos x1) subtract (1.8489550041656857 power (sin x1))))', np.float64(4324967.69304418))
(<__main__.Node object at 0x000002073912ACE0>, '(1.0418552103307164 add (sin (cos x1)))', np.float64(4450879.9165222775))
(<__main__.Node object at 0x000002073912ACE0>, '(1.848

In [ ]:
"""
#### save state problem 
import pickle
# Save to file
with open("population_variable_problem_4.pkl", "wb") as f:
    pickle.dump(POP, f)
"""

In [ ]:
v = POP[0]
print(v)